In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/focusedpico1/processed_data.csv
/kaggle/input/focusedpico1/pubmed_custom_picobert.txt
/kaggle/input/focusedpico1/processed_data_custom (1).csv
/kaggle/input/pubmeddata/processed_data.csv
/kaggle/input/pubmeddata/preprocessed_df_complete.xlsx
/kaggle/input/pubmeddata/pubmed_custom_picobert.txt
/kaggle/input/pubmeddata/pubmed_data.txt
/kaggle/input/pubmeddata/processed_data_custom (3).csv


In [2]:
import re

In [3]:
df=pd.read_csv('/kaggle/input/focusedpico1/processed_data_custom (1).csv')
print(df.head())

   Unnamed: 0      PMID                  PICO_Entity AOMRC_Entity  \
0           0  27932540                          NaN          NaN   
1           1  27932540                    OBJECTIVE            A   
2           2  27932540                 PARTICIPANTS            P   
3           3  27932540                INTERVENTIONS            I   
4           4  27932540  INTERVENTION AND COMPARISON            I   

                                                Text  
0                                                NaN  
1  on a reduction in the incidence of childhood o...  
2                         In school-age children .\n  
3  what is the effect of a school-based physical ...  
4                  compared with no intervention .\n  


In [4]:
df['PICO_Entity'].unique()

array([nan, 'OBJECTIVE', 'PARTICIPANTS', 'INTERVENTIONS',
       'INTERVENTION AND COMPARISON'], dtype=object)

In [5]:
def replace_entity(old_entity):
    if "POPULATION" in old_entity or "SAMPLE" in old_entity:
        entity = "I-POPULATION"
    elif "PARTICIPANT" in old_entity:
        entity = "I-PARTICIPANT"
    elif "INTERVENTION" in old_entity:
        entity = "I-INTERVENTION"
    elif "COMPAR" in old_entity:
        entity = "I-COMPARISON"
    elif "OUTCOME" in old_entity:
        entity = "I-OUTCOME"
    elif bool(re.search('[a-zA-Z]', old_entity)) == False:
        entity = ""
    else:
        entity = "O"
    return(entity)

In [6]:
df.shape

(60, 5)

In [7]:
df = df[pd.notnull(df['Text'])]

In [8]:
df["PICO_Entity"] = df["PICO_Entity"].apply(lambda x: replace_entity(x))

In [9]:
df["Text"] = df["Text"].astype(str)
df["Text"] = df["Text"].apply(lambda x:x.strip())
df = df[["PMID", "Text", "PICO_Entity"]]
df.rename(columns = {'Text':'tokens', 'PICO_Entity':'ner_tags'}, inplace = True)
df = df.reset_index(drop=True)

In [10]:
df.head()

,PMID,tokens,ner_tags
0,27932540,on a reduction in the incidence of childhood o...,O
1,27932540,In school-age children .,I-PARTICIPANT
2,27932540,what is the effect of a school-based physical ...,I-INTERVENTION
3,27932540,compared with no intervention .,I-INTERVENTION
4,27932541,on a reduction in reported incidences of bully...,O


In [11]:
df = df.drop('PMID', axis=1)

In [12]:
print(df['ner_tags'].unique())

['O' 'I-PARTICIPANT' 'I-INTERVENTION']


In [13]:
df.to_csv('converted_input.csv')